In [1]:
#To confirm what we really need...
import pandas as pd

import streamlit as st
import pickle

from bs4 import BeautifulSoup
import regex as re
from nltk.stem.porter import PorterStemmer

In [4]:
with open('model_base.pkl', 'rb') as f: #reading bytes
    foo = pickle.load(f)
    
trial_model = foo

In [6]:
def document_words(raw_document):

    document_text = BeautifulSoup(raw_document).get_text()
    
    letters_only = re.sub("[^a-zA-Z0-9]", " ", document_text)
    
    words = letters_only.split()
    words = [w.lower() for w in words]
 
    p_stemmer = PorterStemmer()
    stemmed_words = [p_stemmer.stem(i, to_lowercase=False) for i in words]

    return(" ".join(stemmed_words))

IE insert here the stuff for the streamlit to gather the following:

In [63]:
# user1 = "Where's a good place to eat food?" #Wow, honestly was harder than I thought to get a 0 prediction... Again, worst case scenario is just having a teacher waste their time - much better than the alternative...
user1 = "bro mang you a stanky bagel boi zionist;lul I'm sure after he fired off all those loyal employees he was drinking their blood and tears on Passover #Zionist;AND HE STILL DOESNT HAVE ENOUGH MONY TO FIX HIS UGLY GIANT ZIONIST NOSE???"

user1_logs=user1.split(';')

End of streamlit userinput<br>
<Br>
 Finally, we do the following:

In [78]:
sample = pd.DataFrame({'comments':user1_logs})
sample['cleaned_comments'] = sample['comments'].apply(document_words)

In [87]:
#I'm still not so into this double df being created, but whatever...

preds = trial_model.predict(sample['cleaned_comments'])
preds_prob = trial_model.predict_proba(sample['cleaned_comments'])
sample_pred_comparer = pd.DataFrame({
    #'cleaned_text':X_test
    'text':sample['comments']
    ,'pred':preds
    ,'p_0':preds_prob[:,0]
    ,'p_1':preds_prob[:,1]
    ,'p_2':preds_prob[:,2]
})
sample_pred_comparer['prob'] = sample_pred_comparer[['p_0','p_1','p_2']].max(axis=1)
sample_pred_comparer['1_prob'] = [sample_pred_comparer.loc[i, 'p_1'] if sample_pred_comparer.loc[i, 'pred'] == 0
                                                      else (1-sample_pred_comparer.loc[i, 'p_0'])
                                                      for i in range(sample_pred_comparer.shape[0])]
#The 1 prob is a bit irrelevant at the moment, but eh... never know. I'm still somewhat concerned of people always going below the radar...


#The idea behind the 1_prob is as follows: A statement that's actually a 2 is just as toxic as a 1, so in a way the probability
#that it is AT LEAST a 1 would be p1 +p2.So toof row a predicted 1: There's still a probability that it's actually a 2, so
#we should be able to add that probability back to p_1 to reflect a more solid liklihood that the post is at least a 1 -
#as once again, anything htat's a 2 is also a 1. Hence, add them...

sample_pred_comparer.sort_values('prob', ascending=False).sort_values('pred', ascending=False).loc[:,["text","pred","prob"]]

,text,pred,prob
1,lul I'm sure after he fired off all those loya...,2,0.649729
2,AND HE STILL DOESNT HAVE ENOUGH MONY TO FIX HI...,1,0.687045
0,bro mang you a stanky bagel boi zionist,1,0.685333


In [71]:
def prob_scanner_account(df, thresh_2=.58, thresh_1=.94, one_response=3):
    overall = 1
    try: #In case a profile doesn't have any 2s... s to with the 1s later on.
        count_2s = df['pred'].value_counts()[2]
        thresh_2 -= .05 * count_2s #Eh, could have simplified in one line, but whatever.
        if df[df['prob']==df['prob'].max()].iloc[0]['p_2'] > thresh_2 and df[df['prob']==df['prob'].max()].iloc[0]['pred'] == 2:
            return "calling the principal AND your guardians"
    except:
        pass
    count_1s = 0
    try:
        count_1s += df['pred'].value_counts()[1]
    except:
        pass
    try:
        count_1s += df['pred'].value_counts()[2]
    except:
        pass
    thresh_1 -= .03 * count_1s
    if df[df['pred']!=0]['1_prob'].mean() > thresh_1 and count_1s >= one_response:
        return "calling the principal"
    else:
        return "free...for now"

In [72]:
prob_scanner_account(sample_pred_comparer)

'calling the principal'

In [27]:
df = sample_pred_comparer.copy()
df.head()

,pred,p_0,p_1,p_2,max_prob,1_prob
0,1,0.138888,0.685333,0.175779,0.685333,0.861112
1,2,0.133519,0.216752,0.649729,0.649729,0.649729
2,1,0.150192,0.687045,0.162763,0.687045,0.849808


In [ ]:
# python -m streamlit run app_malevonce.py

In [ ]:
# streamlit run app_malevonce.py

Ultimately re. balancing the account: We'll still go multi-class with 2s yet essentially go binary with 1s, including the probablity that it's more dangerous then it actually is with a given 1 identification (similaily re 2s (that the formula wasn't confident enough to call out already)). However, we'll account for potential pitfalls by having a sliding scale in the confidence required to summon a teacher(s) to review the social media posts. Such numbers, .05 and .03 (and on that note a one_response of 3 before making an action) are somewhat arbitrary.

One can argue that a slight weight should be added to at least the cases where the model identifid a post as a 2 yet went underneath the initial radar that those probabilities that it's indeed a 2 shouldn't be banally added to the p_1 - but with a scalar of some kind. In short, per philosophies I am influnced by, once we bring down a a stricter thing to something more lenient thing to combin to the volume needed to activate said lenient thing, we treat it entirely as a more lenient thing. Hence it's more 'toxic' nature is ignored and merely nominally considered to be a p_1 via p_1 + p_2.